In [ ]:
%cd ../../spell_correction

In [2]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-xl")
tokenizer = T5Tokenizer.from_pretrained("google/mt5-xl")
for p in model.parameters():
    p.requires_grad = False

## Multilingualなsubword分割がどれくらい日本語で使えそうか
* "平安"や"申し上げ"などが分割されているあたり、ある程度しっかり日本語対応できている（以前のMulti-lingual BERTではほぼcharベースだった）
* 例外として"すもももももももものうち"は"もも"で分割されてしまうので意味的にはあまり良くない分割になっている

In [3]:
'", "'.join(tokenizer.tokenize('すもももももももものうち、とは平安時代に徳川家康が申し上げたものであるといわれている。'))

'▁", "す", "もも", "もも", "もも", "もも", "のうち", "、", "とは", "平安", "時代に", "徳", "川", "家", "康", "が", "申し上げ", "た", "ものである", "といわれ", "ている", "。'

In [4]:
'", "'.join(tokenizer.tokenize("本記事では Google の T5(Text-to-Text Transfer Transformer) *1によるテキスト生成について、学習や推論のコード例と実験結果を交えてご紹介します。実験としては livedoor ニュースコーパス*2での文章分類、やさしい日本語コーパス*3及びやさしい日本語拡張コーパス*4を用いたやさしい日本語変換を行いました。"))

'▁", "本", "記事", "では", "▁Google", "▁", "の", "▁T", "5", "(", "Text", "-", "to", "-", "Text", "▁Transfer", "▁Transformer", ")", "▁", "*1", "による", "テキスト", "生成", "について", "、", "学習", "や", "推", "論", "の", "コード", "例", "と", "実験", "結果を", "交", "えて", "ご紹介します", "。", "実験", "としては", "▁live", "door", "▁", "ニュース", "コー", "パス", "*", "2", "での", "文章", "分類", "、", "やさしい", "日本語", "コー", "パス", "*", "3", "及び", "やさしい", "日本語", "拡張", "コー", "パス", "*", "4", "を用いた", "やさしい", "日本語", "変換", "を行いました", "。'

## mT5-xlの言語モデリングの性能をチェック

In [5]:
text = "安倍晋三は日本の <extra_id_0> だ。"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 'リーダー',
 '▁<extra_id_1>',
 '▁',
 '国',
 '▁<extra_id_2>',
 'リーダー',
 '▁<extra_id_3>',
 '▁',
 'だ',
 '。',
 '▁<extra_id_4>',
 'リーダー',
 '▁<extra_id_5>',
 'リーダー',
 '▁<extra_id_6>',
 'リーダー',
 '▁<extra_id_7>',
 'リーダー']

In [6]:
text = "安倍晋三は日本の <extra_id_0> で、 <extra_id_1> と呼ばれる経済政策を打ち出し日本経済を大きく回復させた。"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 '首相',
 '▁<extra_id_1>',
 'アベ',
 'ノミ',
 'クス',
 '▁<extra_id_2>',
 '首相',
 '▁<extra_id_3>',
 'アベ',
 'ノミ',
 'クス',
 '▁<extra_id_4>',
 '首相',
 '▁<extra_id_5>',
 '首相',
 '▁<extra_id_6>',
 'アベ',
 'ノミ']

In [7]:
# ❌な例, wiki
text = " <extra_id_0>はグラフ理論における最短経路問題を解くための最良優先探索によるアルゴリズムである。"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 '。',
 '▁',
 '本',
 'アル',
 'ゴ',
 'リズム',
 '▁<extra_id_1>',
 '、',
 '一般的に',
 '▁<extra_id_2>',
 'であり',
 '、',
 '一般的に',
 '▁<extra_id_3>',
 'である',
 '。',
 '▁',
 '本']

In [8]:
# ⭕️な例, yahoo news
text = "売れっ子 <extra_id_0>だけに、豪邸で優雅な暮らしをしているのでは？　と思いきや、 <extra_id_1>を送っている。"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 '俳優',
 '▁<extra_id_1>',
 '実は',
 '普通の',
 '生活',
 '▁<extra_id_2>',
 '俳優',
 'な',
 '▁<extra_id_3>',
 '実は',
 '普通の',
 '生活',
 '▁<extra_id_4>',
 '実は',
 '普通の',
 '生活',
 '▁<extra_id_5>',
 '俳優']

In [9]:
# ⭕️, 5ch
# "ｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗ ゴミゲーを１０円で買ったからって１０円の損失やってのｗオマエｗ分かってねえなｗｗｗｗｗ"
text = "ｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗ ゴミゲーを１０円で買った <extra_id_0>１０円の損失やってのｗ <extra_id_1>ｗ分かってねえな <extra_id_2>"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 'のに',
 '▁<extra_id_1>',
 '▁',
 '馬鹿',
 '▁<extra_id_2>',
 'w',
 '</s>']

In [10]:
# ⭕️, twitter
# "しやがれ中ずっと 泣くもんかって涙堪えてたんだけど、HELLO NEW DREAMのcm流れてダメだった。素敵なサプライズ....もう愛しかなくて....すごいよ嵐。スポンサーの皆さんありがとう号泣"
text = "しやがれ中ずっと 泣くもんかって涙堪えてたんだけど、HELLO NEW DREAMのcm流れて <extra_id_0>だった。素敵な <extra_id_1>もう愛しかなくて....すごいよ嵐。スポンサーの皆さん <extra_id_2>号泣"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 '▁',
 '涙',
 'が',
 '溢れ',
 'そう',
 '▁<extra_id_1>',
 '曲',
 'だね',
 '。',
 '▁<extra_id_2>',
 'ありがとう',
 '。',
 '▁<extra_id_3>',
 '▁',
 '▁<extra_id_4>',
 '▁',
 '涙',
 'が']

# mT5でzero-shotなスペル修正を行う
検証すること
1. Recallの高いエラー検出ができたとして、「decodingの各ステップで編集距離による制約を満たす最も尤度の高いトークンを選択していく」ことでエラー修正ができそうかどうかを確認
2. mT5のdecoderを使ってスペルエラー検出ができそうかどうか

## 1. エラー検出ができたとして、エラー修正ができそうかどうか

In [11]:
texts = [
    "彼はファヴローにクラッシック西部劇映画を提供した。", # 'クラッシック' → 'クラシック'
    "民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位だた。",  # 'だた' → 'だった'
    "クラゲには4つ放射管を持つが触手はもたない。",  # '' -> 'の'
    "桶狭間の戦いで国友制の鉄砲が織田信長により初めて戦力として使用された。",  # '制' → '製'
    "ひきこもりの更正をはじめたきっかけ",  # '正' → '生'
]    

In [12]:
# 何かしらのエラー検出でマスクする
masked_texts = [
    "彼はファヴローに <extra_id_0>西部劇映画を提供した。", # 'クラッシック' → 'クラシック'
    "民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位 <extra_id_0>",  # 'だた' → 'だった'
    "クラゲには <extra_id_0>を持つが触手はもたない。",  # '' -> 'の'
    "桶狭間の戦いで <extra_id_0>の鉄砲が織田信長により初めて戦力として使用された。",  # '制' → '製'
    "ひきこもりの <extra_id_0>をはじめたきっかけ",  # '正' → '生'
]

In [13]:
batch = tokenizer.prepare_seq2seq_batch(src_texts=masked_texts, tgt_texts=['<pad> <extra_id_0>'] * 5, return_tensors="pt")
output_step1 = model(**batch)

In [14]:
top_logits, top_indices = output_step1.logits[:, -1].sort(dim=-1, descending=True)  # 末尾のトークンの次にくる単語の予測をスコアでソート

In [15]:
print(top_logits.shape, top_indices.shape)

torch.Size([5, 250112]) torch.Size([5, 250112])


In [16]:
top_tokens = [tokenizer.convert_ids_to_tokens(each_indices) for each_indices in top_indices.numpy().tolist()]

In [17]:
from spell_correction.utils import BeginningLevenshtein
beginning_levenshtein = BeginningLevenshtein()

In [18]:
import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

### 'クラッシック'に代わる尤もらしい単語

原文: 彼はファヴローにクラッシック西部劇映画を提供した。

* 単純に尤度が高い単語を選ぶと、文意を損なう。
* 尤度が高く、かつ'ク'から始まるトークンを選ぶとスペル修正に使えそう
  * 厳密には'クラッシック'の冒頭部との編集距離が小さく、よりマッチしているトークン（詳細は後述）

In [19]:
original_token = 'クラッシック'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[0], top_logits[0], top_indices[0])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / (candidate['distance'] + 1) * sigmoid(candidate['logit'])
    candidates.append(candidate)

In [20]:
for c in candidates[:30]:
    print(c)

{'match_ref': '', 'match_len': 0, 'distance': 3.0, 'rank': 0, 'token': '多くの', 'logit': 0.07193946838378906, 'index': 36431, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 5.0, 'rank': 1, 'token': 'いくつかの', 'logit': -0.5945250988006592, 'index': 176894, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 1.0, 'rank': 2, 'token': '、', 'logit': -1.3016502857208252, 'index': 292, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 2.0, 'rank': 3, 'token': '彼の', 'logit': -1.5224401950836182, 'index': 104567, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 3.0, 'rank': 4, 'token': '多数の', 'logit': -1.5524663925170898, 'index': 209087, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 3.0, 'rank': 5, 'token': '優れた', 'logit': -1.7015354633331299, 'index': 163464, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 3.0, 'rank': 6, 'token': '最高の', 'logit': -1.9628660678863525, 'index': 125049, 'match_sco

In [21]:
for c in sorted(candidates, key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': 'クラッシック', 'match_len': 6, 'distance': 1.0, 'rank': 232, 'token': 'クラシック', 'logit': -4.78184700012207, 'index': 119003, 'match_score': 0.02493257005628992}
{'match_ref': 'クラ', 'match_len': 2, 'distance': 3.0, 'rank': 220, 'token': 'フランス', 'logit': -4.710697174072266, 'index': 75179, 'match_score': 0.0044591254873828255}
{'match_ref': 'クラッ', 'match_len': 3, 'distance': 4.0, 'rank': 323, 'token': 'ヨーロッパ', 'logit': -5.108414649963379, 'index': 122950, 'match_score': 0.0036055976733741757}
{'match_ref': 'クラッ', 'match_len': 3, 'distance': 2.0, 'rank': 640, 'token': 'ブラック', 'logit': -5.78403377532959, 'index': 21545, 'match_score': 0.0030668468356141723}
{'match_ref': 'クラ', 'match_len': 2, 'distance': 2.0, 'rank': 443, 'token': 'ドラマ', 'logit': -5.399911403656006, 'index': 53130, 'match_score': 0.002997779827072545}
{'match_ref': 'ク', 'match_len': 1, 'distance': 0.0, 'rank': 708, 'token': 'ク', 'logit': -5.884771823883057, 'index': 6662, 'match_score': 0.002773765644475654}
{'matc

In [22]:
# 別のスコア関数を試してみる (distanceにexpをかけることで、編集距離を重視)
# 2番目の候補が'ク'になっていていい感じ
original_token = 'クラッシック'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[0], top_logits[0], top_indices[0])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / math.exp(candidate['distance']) * sigmoid(candidate['logit'])
    candidates.append(candidate)
for c in sorted(candidates, key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': 'クラッシック', 'match_len': 6, 'distance': 1.0, 'rank': 232, 'token': 'クラシック', 'logit': -4.78184700012207, 'index': 119003, 'match_score': 0.018344359878551545}
{'match_ref': 'ク', 'match_len': 1, 'distance': 0.0, 'rank': 708, 'token': 'ク', 'logit': -5.884771823883057, 'index': 6662, 'match_score': 0.002773765644475654}
{'match_ref': 'クラ', 'match_len': 2, 'distance': 0.0, 'rank': 1863, 'token': 'クラ', 'logit': -7.138404369354248, 'index': 48440, 'match_score': 0.0015867761625547956}
{'match_ref': 'クラ', 'match_len': 2, 'distance': 1.0, 'rank': 912, 'token': 'トラ', 'logit': -6.183650493621826, 'index': 56376, 'match_score': 0.0015146603269555882}
{'match_ref': 'クラッ', 'match_len': 3, 'distance': 2.0, 'rank': 640, 'token': 'ブラック', 'logit': -5.78403377532959, 'index': 21545, 'match_score': 0.00124515775542346}
{'match_ref': 'クラ', 'match_len': 2, 'distance': 2.0, 'rank': 443, 'token': 'ドラマ', 'logit': -5.399911403656006, 'index': 53130, 'match_score': 0.0012171161459336001}
{'match_ref'

In [23]:
# "民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位 <extra_id_0>",  # 'だた' → 'だった'
original_token = 'だた'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[0], top_logits[0], top_indices[0])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / (candidate['distance'] + 1) * sigmoid(candidate['logit'])
    candidates.append(candidate)
for c in sorted(candidates, key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': 'だた', 'match_len': 2, 'distance': 2.0, 'rank': 5, 'token': '優れた', 'logit': -1.7015354633331299, 'index': 163464, 'match_score': 0.10284322075251456}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 44, 'token': 'また', 'logit': -3.4567394256591797, 'index': 6134, 'match_score': 0.030568509618185774}
{'match_ref': 'だ', 'match_len': 1, 'distance': 1.0, 'rank': 53, 'token': 'まだ', 'logit': -3.5848755836486816, 'index': 27363, 'match_score': 0.013495687398999649}
{'match_ref': 'だた', 'match_len': 2, 'distance': 2.0, 'rank': 149, 'token': '新たな', 'logit': -4.3365936279296875, 'index': 87989, 'match_score': 0.008608071841784054}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 236, 'token': '似た', 'logit': -4.786910057067871, 'index': 224733, 'match_score': 0.008269231763908105}
{'match_ref': 'だた', 'match_len': 2, 'distance': 3.0, 'rank': 263, 'token': '発表した', 'logit': -4.878444671630859, 'index': 193782, 'match_score': 0.0037756909117377113}
{'match_ref': 'だた

In [24]:
# 別のスコア関数を試してみる (distanceにexpをかけることで、編集距離を重視)
# "民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位 <extra_id_0>",  # 'だた' → 'だった'
original_token = 'だた'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[0], top_logits[0], top_indices[0])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / math.exp(candidate['distance']) * sigmoid(candidate['logit'])
    candidates.append(candidate)
for c in sorted(candidates, key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': 'だた', 'match_len': 2, 'distance': 2.0, 'rank': 5, 'token': '優れた', 'logit': -1.7015354633331299, 'index': 163464, 'match_score': 0.04175494922852113}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 44, 'token': 'また', 'logit': -3.4567394256591797, 'index': 6134, 'match_score': 0.022491052471564084}
{'match_ref': 'だ', 'match_len': 1, 'distance': 1.0, 'rank': 53, 'token': 'まだ', 'logit': -3.5848755836486816, 'index': 27363, 'match_score': 0.009929571877136934}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 236, 'token': '似た', 'logit': -4.786910057067871, 'index': 224733, 'match_score': 0.0060841607204473085}
{'match_ref': 'だた', 'match_len': 2, 'distance': 2.0, 'rank': 149, 'token': '新たな', 'logit': -4.3365936279296875, 'index': 87989, 'match_score': 0.003494927522486866}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 891, 'token': '見た', 'logit': -6.165638446807861, 'index': 113704, 'match_score': 0.0015421319443203903}
{'match_ref': 'だ',

In [25]:
# 別のスコア関数を試してみる (distanceにexpをかけることで、編集距離を重視)
# "民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位 <extra_id_0>",  # 'だた' → 'だった'
original_token = 'だた'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[0], top_logits[0], top_indices[0])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / math.exp(candidate['distance']) * sigmoid(candidate['logit'])
    candidates.append(candidate)

In [26]:
for c in sorted(filter(lambda c: c['distance'] <= 1, candidates), key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 44, 'token': 'また', 'logit': -3.4567394256591797, 'index': 6134, 'match_score': 0.022491052471564084}
{'match_ref': 'だ', 'match_len': 1, 'distance': 1.0, 'rank': 53, 'token': 'まだ', 'logit': -3.5848755836486816, 'index': 27363, 'match_score': 0.009929571877136934}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 236, 'token': '似た', 'logit': -4.786910057067871, 'index': 224733, 'match_score': 0.0060841607204473085}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 891, 'token': '見た', 'logit': -6.165638446807861, 'index': 113704, 'match_score': 0.0015421319443203903}
{'match_ref': 'だ', 'match_len': 1, 'distance': 1.0, 'rank': 517, 'token': 'だけ', 'logit': -5.551756381988525, 'index': 13386, 'match_score': 0.001422087371044277}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 2571, 'token': 'わた', 'logit': -7.621779441833496, 'index': 216460, 'match_score': 0.00036010251212698915}
{'match_ref': 'だた